# FAST: Energy Benchmarking

In [65]:
import torch
import subprocess
import re
import time

In [3]:
device_name = "cpu"  # Default device is CPU
if torch.cuda.is_available():
    # I read that this works for detecting if notebook is being run in a colab environment, not sure though
    if 'COLAB_GPU' in os.environ:
        print("colab environment detected")
        device_name = "gpu" 
    else:
        device_name = "cuda" # CUDA for NVIDIA GPU
elif torch.backends.mps.is_available():
    device_name = torch.device("mps")  # Metal Performance Shaders for Apple M-series GPU

# device_name = "cuda:0"
device = torch.device(device_name)
print(f"Device: {device_name}")

Device: mps


In [81]:
def match_regex(output, pattern):
    match = pattern.search(output, re.MULTILINE)
    if match:
        return float(match.group(1))
    else:
        raise ValueError("Regex not found in output")

In [82]:
def get_power_mW(verbose=False):
    try:
        output = subprocess.check_output(["sudo", "powermetrics", "-n", "1", "-i", "1000", "--samplers", "all"], universal_newlines=True)  # Suppress output
        gpu_power = match_regex(output, re.compile(r"GPU Power: (\d+) mW"))
        total_cpu_power = match_regex(output, re.compile(r"CPU Power: (\d+) mW"))
        cpu_time  = match_regex(output, re.compile(r"python3\.9\s+\d+\s+(\d+\.\d+)"))  # assumes we are on python3.9
        total_cpu_time  = match_regex(output, re.compile(r"ALL_TASKS\s+\-2+\s+(\d+\.\d+)"))  # assumes we are on python3.9

        scaled_cpu_power = total_cpu_power * cpu_time / total_cpu_time
        if verbose:
            print(f"Total CPU Power: {total_cpu_power} mW")
            print(f"CPU Time: {cpu_time} ms")
            print(f"Total CPU Time: {total_cpu_time} ms")
            print(f"Scaled CPU Power: {scaled_cpu_power} mW")
            print(f"GPU Power: {gpu_power} mW")
            print(f"Power for python3.9: {scaled_cpu_power + gpu_power} mW")
            # write output to file
            with open("power.out", "w") as f:
                f.write(str(output))

        return scaled_cpu_power + gpu_power
    
    except Exception as e:
        print(f"Error: {e}")

In [83]:
def poll_power(values, period=5, verbose=False):
    try:
        while True:
            power = get_power_mW(verbose=verbose)
            values.append(power)
            print(power)
            time.sleep(period)
    except Exception as e:
        print(f"Error: {e}")

In [86]:
power_values = []
poll_power(power_values, period=3, verbose=True)

Total CPU Power: 7943.0 mW
CPU Time: 873.78 ms
Total CPU Time: 3566.04 ms
Scaled CPU Power: 1946.2581855503583 mW
GPU Power: 3636.0 mW
Power for python3.9: 5582.258185550359 mW
5582.258185550359


Second underflow occured.


Total CPU Power: 6604.0 mW
CPU Time: 859.26 ms
Total CPU Time: 2989.29 ms
Scaled CPU Power: 1898.2945916923418 mW
GPU Power: 3521.0 mW
Power for python3.9: 5419.294591692342 mW
5419.294591692342


KeyboardInterrupt: 